<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/cert_prof_dl_intro/4%20-%20Usando%20imagens%20do%20mundo%20real/16%20-%20C1_W4_Lab_2_image_generator_with_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Certificado Profissional Desenvolvedor do TensorFlow](https://www.coursera.org/professional-certificates/tensorflow-in-practice) de [Laurence Moroney](https://laurencemoroney.com/)

# ImageDataGenerator Com conjunto de Validação

Neste laboratório, você continuará a usar a classe `ImageDataGenerator` para preparar o conjunto de dados `Horses or Humans`.

Desta vez, você adicionará um conjunto de validação para que também possa medir o desempenho do modelo em dados que ele ainda não viu.

Execute os blocos de código abaixo para fazer download dos conjuntos de dados `horse-or-human.zip` e `validation-horse-or-human.zip`, respectivamente.

In [ ]:
# Faça o download do conjunto de treinamento
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip

In [ ]:
# Baixar o conjunto de validação
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip

Em seguida, descompacte os dois arquivos.

In [ ]:
import zipfile

# Descompactar o conjunto de treinamento
local_zip = './horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./horse-or-human')

# Descompactar o conjunto de validação
local_zip = './validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./validation-horse-or-human')

zip_ref.close()

Da mesma forma que no laboratório anterior, você definirá os diretórios que contêm as imagens.

Desta vez, você incluirá aqueles com dados de validação.

In [ ]:
import os

# Diretório com fotos de cavalos de treinamento
train_horse_dir = os.path.join('./horse-or-human/horses')

# Diretório com imagens humanas de treinamento
train_human_dir = os.path.join('./horse-or-human/humans')

# Diretório com imagens de cavalos de validação
validation_horse_dir = os.path.join('./validation-horse-or-human/horses')

# Diretório com imagens humanas de validação
validation_human_dir = os.path.join('./validation-horse-or-human/humans')

Agora veja como são os nomes dos arquivos nesses diretórios:

In [ ]:
train_horse_names = os.listdir(train_horse_dir)
print(f'CONJUNTO DE TREINO, CAVALOS: {train_horse_names[:10]}')

train_human_names = os.listdir(train_human_dir)
print(f'CONJUNTO DE TREINO, HUMANOS: {train_human_names[:10]}')

validation_horse_names = os.listdir(validation_horse_dir)
print(f'CONJUNTO DE VALIDAÇÃO, CAVALOS: {validation_horse_names[:10]}')

validation_human_names = os.listdir(validation_human_dir)
print(f'CONJUNTO DE VALIDAÇÃO, HUMANOS: {validation_human_names[:10]}')

Você pode descobrir o número total de imagens de cavalos e humanos nos diretórios:

In [ ]:
print(f'total de imagens de treinamento de cavalos: {len(os.listdir(train_horse_dir))}')
print(f'total de imagens de treinamento de humanos: {len(os.listdir(train_human_dir))}')
print(f'total de imagens de validação de cavalos: {len(os.listdir(validation_horse_dir))}')
print(f'total de imagens de validação de humanos: {len(os.listdir(validation_human_dir))}')

Agora, dê uma olhada em algumas imagens para ter uma noção melhor de como elas são. Primeiro, configure os parâmetros do `matplotlib`:

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Parâmetros para nosso gráfico; produziremos imagens em uma configuração 4x4
nrows = 4
ncols = 4

# Índice para iteração de imagens
pic_index = 0

Agora, exiba um lote de 8 imagens de cavalos e 8 imagens de humanos. Você pode executar novamente a célula para ver um novo lote a cada vez:

In [ ]:
# Configure o matplotlib fig e dimensione-o para caber em fotos 4x4
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
next_horse_pix = [os.path.join(train_horse_dir, fname) 
                for fname in train_horse_names[pic_index-8:pic_index]]
next_human_pix = [os.path.join(train_human_dir, fname) 
                for fname in train_human_names[pic_index-8:pic_index]]

for i, img_path in enumerate(next_horse_pix+next_human_pix):
  # Configure o subplot; os índices do subplot começam em 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Não mostre os eixos (ou linhas de grade)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()


## Criando um modelo pequeno a partir do zero

Você definirá a mesma arquitetura de modelo anterior:

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # Observe que a forma de entrada é o tamanho desejado da imagem 300x300 com 3 bytes de cor
    # Essa é a primeira convolução
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # A segunda convolução
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # A terceira convolução
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # A quarta convolução
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # A quinta convolução
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Achatar os resultados para alimentar uma DNN
    tf.keras.layers.Flatten(),
    # Camada oculta de 512 neurônios
    tf.keras.layers.Dense(512, activation='relu'),
    # Apenas 1 neurônio de saída. Ele conterá um valor de 0 a 1, sendo 0 para uma classe ("cavalos") e 1 para a outra ("humanos")
    tf.keras.layers.Dense(1, activation='sigmoid')
])

Você pode revisar a arquitetura da rede e as formas de saída com `model.summary()`.

In [ ]:
model.summary()

Você também usará as mesmas configurações de compilação de antes:

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

### Pré-processamento de dados

Agora você configurará os geradores de dados.

Em geral, serão parecidos com os anteriores, mas observe o código adicional para preparar também os dados de validação. 

Ele precisará ser instanciado separadamente e também dimensionado para ter um intervalo de `[0,1]` de valores de pixel.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Todas as imagens serão redimensionadas em 1,/255
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# Imagens de treinamento de fluxo em lotes de 128 usando o gerador train_datagen
train_generator = train_datagen.flow_from_directory(
        './horse-or-human/',  # Esse é o diretório de origem das imagens de treinamento
        target_size=(300, 300),  # Todas as imagens serão redimensionadas para 300x300
        batch_size=128,
        # Como você usa a perda binary_crossentropy, precisa de rótulos binários
        class_mode='binary')

# Imagens de validação de fluxo em lotes de 128 usando o gerador validation_datagen
validation_generator = validation_datagen.flow_from_directory(
        './validation-horse-or-human/',  # Esse é o diretório de origem das imagens de validação
        target_size=(300, 300),  # Todas as imagens serão redimensionadas para 300x300
        batch_size=32,
        # Como você usa a perda binary_crossentropy, precisa de rótulos binários
        class_mode='binary')

### Treinamento

Agora, treine o modelo por 15 épocas. Aqui, você passará parâmetros para `validation_data` e `validation_steps`.

Com eles, você notará saídas adicionais nas instruções de impressão: `val_loss` e `val_accuracy`.

Observe que, à medida que você treina com mais épocas, a precisão do treinamento pode aumentar, mas a precisão da validação diminui.

Isso pode ser um sinal de ajuste excessivo e você precisa evitar que seu modelo chegue a esse ponto.

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8)

### Previsão do modelo

Agora, dê uma olhada na execução de uma previsão usando o modelo. Esse código permitirá que você escolha um ou mais arquivos do seu sistema de arquivos, carregue-os e execute-os por meio do modelo, fornecendo uma indicação de que o objeto é um cavalo ou um humano.

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded = files.upload()

for fn in uploaded.keys():
 
  # Previsão de imagens
  path = '/content/' + fn
  img = load_img(path, target_size=(300, 300))
  x = img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " é um humano")
  else:
    print(fn + " é um cavalo")
 

### Visualizando representações intermediárias

Como antes, você pode plotar como os recursos são transformados à medida que passam por cada camada.

In [ ]:
import numpy as np
import random
from tensorflow.keras.utils import img_to_array, load_img

# Defina um novo modelo que receberá uma imagem como entrada e produzirá
# representações intermediárias para todas as camadas do modelo anterior após
# a primeira.
successive_outputs = [layer.output for layer in model.layers[1:]]
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)

# Prepare uma imagem de entrada aleatória do conjunto de treinamento.
horse_img_files = [os.path.join(train_horse_dir, f) for f in train_horse_names]
human_img_files = [os.path.join(train_human_dir, f) for f in train_human_names]
img_path = random.choice(horse_img_files + human_img_files)

img = load_img(img_path, target_size=(300, 300))  # esta é uma imagem PIL
x = img_to_array(img)  # Matriz Numpy com formato (300, 300, 3)
x = x.reshape((1,) + x.shape)  # Matriz Numpy com formato (1, 300, 300, 3)

# Escale em 1/255
x /= 255

# Execute a imagem na rede, obtendo assim todas as
# representações intermediárias para essa imagem.
successive_feature_maps = visualization_model.predict(x)

# Esses são os nomes das camadas, para que você possa tê-las como parte do gráfico
layer_names = [layer.name for layer in model.layers[1:]]

# Exibir as representações
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  if len(feature_map.shape) == 4:

    # Faça isso apenas para as camadas conv / maxpool, não para as camadas totalmente conectadas
    n_features = feature_map.shape[-1]  # number of features in feature map

    # O mapa de recursos tem a forma (1, tamanho, tamanho, n_recursos)
    size = feature_map.shape[1]
    
    # Colocar as imagens em mosaico nessa matriz
    display_grid = np.zeros((size, size * n_features))
    for i in range(n_features):
      x = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std()
      x *= 64
      x += 128
      x = np.clip(x, 0, 255).astype('uint8')
    
      # Colocar cada filtro em uma grade horizontal grande
      display_grid[:, i * size : (i + 1) * size] = x
    
    # Exibir a grade
    scale = 20. / n_features
    plt.figure(figsize=(scale * n_features, scale))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

## Limpar

Antes de executar o próximo exercício, execute a seguinte célula para encerrar o kernel e liberar recursos de memória:

In [ ]:
from google.colab import runtime
runtime.unassign()